In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from scipy.spatial import distance
from tqdm import tqdm
import timm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 1. 加载 CIFAR-10 数据集
transform = transforms.Compose([
    transforms.Resize((224, 224)), # 调整为预训练模型所需的尺寸
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=False)

test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

# 2. 使用 TIMM 预训练模型进行降维
model = timm.create_model('resnet50', pretrained=True)
model = model.to(device).eval() # 移动模型到GPU并设置为评估模式

# 为了提取倒数第二个全连接层的输出，我们需要删除最后的分类层
model = torch.nn.Sequential(*(list(model.children())[:-1]))

def extract_features(dataloader):
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Extracting features"):
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.squeeze(-1).squeeze(-1).cpu().numpy())
            labels.append(targets.numpy())
    return np.vstack(features), np.concatenate(labels)

train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

# 3. 使用最小 Mahalanobis 距离分类器进行分类
class_means = {}
class_cov_matrices = {}
classes = np.unique(train_labels)

for c in classes:
    class_data = train_features[train_labels == c]
    class_means[c] = np.mean(class_data, axis=0)
    class_cov_matrices[c] = np.cov(class_data, rowvar=False)

def mahalanobis_distance(x, mean, cov_matrix):
    return distance.mahalanobis(x, mean, np.linalg.inv(cov_matrix))

def classify_sample(x):
    distances = {}
    for c in classes:
        distances[c] = mahalanobis_distance(x, class_means[c], class_cov_matrices[c])
    return min(distances, key=distances.get)

train_pred = np.array([classify_sample(x) for x in tqdm(train_features, desc="Classifying train samples")])
test_pred = np.array([classify_sample(x) for x in tqdm(test_features, desc="Classifying test samples")])

# 计算训练和测试的分类准确率
train_accuracy = np.mean(train_pred == train_labels)
test_accuracy = np.mean(test_pred == test_labels)
print(f"Training accuracy using minimum Mahalanobis distance classifier after feature extraction: {train_accuracy:.4f}")
print(f"Test accuracy using minimum Mahalanobis distance classifier after feature extraction: {test_accuracy:.4f}")


100%|██████████| 170498071/170498071 [00:03<00:00, 43557307.67it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Classifying train samples:   0%|          | 0/50000 [00:00<?, ?it/s]

LinAlgError: ignored

In [ ]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from scipy.spatial import distance
from tqdm import tqdm
import timm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 1. 加载 CIFAR-10 数据集
transform = transforms.Compose([
    transforms.Resize((224, 224)), # 调整为预训练模型所需的尺寸
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=False)

test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=False)

# 2. 使用 TIMM 预训练模型进行降维
model = timm.create_model('resnet50', pretrained=True)
model = model.to(device).eval()

# 为了提取倒数第二个全连接层的输出，我们需要删除最后的分类层
model = torch.nn.Sequential(*(list(model.children())[:-1]))

def extract_features(dataloader):
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Extracting features"):
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.squeeze(-1).squeeze(-1).cpu().numpy())
            labels.append(targets.numpy())
    return np.vstack(features), np.concatenate(labels)

train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

# 3. 使用最小 Mahalanobis 距离分类器进行分类
class_means = {}
class_cov_matrices = {}
classes = np.unique(train_labels)

regularization_factor = 1e-6
for c in classes:
    class_data = train_features[train_labels == c]
    class_means[c] = np.mean(class_data, axis=0)
    class_cov_matrices[c] = np.cov(class_data, rowvar=False) + np.eye(class_data.shape[1]) * regularization_factor

def mahalanobis_distance(x, mean, cov_matrix):
    return distance.mahalanobis(x, mean, np.linalg.inv(cov_matrix))

def classify_sample(x):
    distances = {}
    for c in classes:
        distances[c] = mahalanobis_distance(x, class_means[c], class_cov_matrices[c])
    return min(distances, key=distances.get)

train_pred = np.array([classify_sample(x) for x in tqdm(train_features, desc="Classifying train samples")])
test_pred = np.array([classify_sample(x) for x in tqdm(test_features, desc="Classifying test samples")])

# 计算训练和测试的分类准确率
train_accuracy = np.mean(train_pred == train_labels)
test_accuracy = np.mean(test_pred == test_labels)
print(f"Training accuracy using minimum Mahalanobis distance classifier after feature extraction: {train_accuracy:.4f}")
print(f"Test accuracy using minimum Mahalanobis distance classifier after feature extraction: {test_accuracy:.4f}")


Files already downloaded and verified
Files already downloaded and verified



Extracting features: 100%|██████████| 98/98 [04:03<00:00,  2.48s/it]

Extracting features: 100%|██████████| 20/20 [00:48<00:00,  2.43s/it]

Classifying train samples:   0%|          | 15/50000 [01:55<106:00:54,  7.64s/it]

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from scipy.spatial import distance
from tqdm import tqdm
import timm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 1. 加载 CIFAR-10 数据集
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整为预训练模型所需的尺寸
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=False)

test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=False)

# 2. 使用 TIMM 预训练模型进行降维
desired_dim = 512  # 设置你想要的维度

# 定义一个新的线性层进行降维
dim_reduction_layer = torch.nn.Linear(2048, desired_dim).to(device)

# 将其与原始模型连接在一起
model = torch.nn.Sequential(
    *(list(timm.create_model('resnet50', pretrained=True).children())[:-1]),  # 原始的resnet50，但去除最后的分类层
    torch.nn.Flatten(1),  # 压扁张量，使其变为2D
    dim_reduction_layer  # 进行降维
).to(device).eval()

def extract_features(dataloader):
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Extracting features"):
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())
    return np.vstack(features), np.concatenate(labels)

train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

# 3. 使用最小 Mahalanobis 距离分类器进行分类
class_means = {}
class_cov_matrices = {}
classes = np.unique(train_labels)

regularization_factor = 1e-6
for c in classes:
    class_data = train_features[train_labels == c]
    class_means[c] = np.mean(class_data, axis=0)
    class_cov_matrices[c] = np.cov(class_data, rowvar=False) + np.eye(class_data.shape[1]) * regularization_factor

def mahalanobis_distance(x, mean, cov_matrix):
    return distance.mahalanobis(x, mean, np.linalg.inv(cov_matrix))

def classify_sample(x):
    distances = {}
    for c in classes:
        distances[c] = mahalanobis_distance(x, class_means[c], class_cov_matrices[c])
    return min(distances, key=distances.get)

train_pred = np.array([classify_sample(x) for x in tqdm(train_features, desc="Classifying train samples")])
test_pred = np.array([classify_sample(x) for x in tqdm(test_features, desc="Classifying test samples")])

# 计算训练和测试的分类准确率
train_accuracy = np.mean(train_pred == train_labels)
test_accuracy = np.mean(test_pred == test_labels)
print(f"Training accuracy using minimum Mahalanobis distance classifier after feature extraction: {train_accuracy:.4f}")
print(f"Test accuracy using minimum Mahalanobobis distance classifier after feature extraction: {test_accuracy:.4f}")


Files already downloaded and verified
Files already downloaded and verified




Extracting features:   0%|          | 0/98 [00:00<?, ?it/s]

Extracting features:   1%|          | 1/98 [00:02<03:48,  2.36s/it]

Extracting features:   2%|▏         | 2/98 [00:04<03:42,  2.32s/it]

Extracting features:   3%|▎         | 3/98 [00:06<03:39,  2.31s/it]

Extracting features:   4%|▍         | 4/98 [00:09<03:35,  2.29s/it]

Extracting features:   5%|▌         | 5/98 [00:11<03:33,  2.29s/it]

Extracting features:   6%|▌         | 6/98 [00:14<03:41,  2.41s/it]

Extracting features:   7%|▋         | 7/98 [00:16<03:35,  2.37s/it]

Extracting features:   8%|▊         | 8/98 [00:18<03:30,  2.34s/it]

Extracting features:   9%|▉         | 9/98 [00:21<03:27,  2.33s/it]

Extracting features:  10%|█         | 10/98 [00:23<03:24,  2.32s/it]

Extracting features:  11%|█         | 11/98 [00:25<03:29,  2.41s/it]

Extracting features:  12%|█▏        | 12/98 [00:28<03:24,  2.37s/it]

Extracting features:  13%|█▎        | 13/98 [00:30<03:19,  2.35s/it]

Extracting features:  14%|█▍        

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from scipy.spatial import distance
from tqdm import tqdm
import timm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 1. 加载 CIFAR-10 数据集
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整为预训练模型所需的尺寸
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=False)

test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=512, shuffle=False)

# 2. 使用 ResNet-50 的 layer2 作为特征提取层
model = timm.create_model('resnet50', pretrained=True).to(device)
children_list = list(model.children())
selected_layer = torch.nn.Sequential(*children_list[:-4])
gap = torch.nn.AdaptiveAvgPool2d(1)
feature_extractor = torch.nn.Sequential(selected_layer, gap, torch.nn.Flatten()).to(device).eval()

def extract_features(dataloader):
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Extracting features"):
            inputs = inputs.to(device)
            outputs = feature_extractor(inputs)
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())
    return np.vstack(features), np.concatenate(labels)

train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

# 3. 使用最小 Mahalanobis 距离分类器进行分类
class_means = {}
class_cov_matrices = {}
classes = np.unique(train_labels)

regularization_factor = 1e-6
for c in classes:
    class_data = train_features[train_labels == c]
    class_means[c] = np.mean(class_data, axis=0)
    class_cov_matrices[c] = np.cov(class_data, rowvar=False) + np.eye(class_data.shape[1]) * regularization_factor

def mahalanobis_distance(x, mean, cov_matrix):
    return distance.mahalanobis(x, mean, np.linalg.inv(cov_matrix))

def classify_sample(x):
    distances = {}
    for c in classes:
        distances[c] = mahalanobis_distance(x, class_means[c], class_cov_matrices[c])
    return min(distances, key=distances.get)

train_pred = np.array([classify_sample(x) for x in tqdm(train_features, desc="Classifying train samples")])
test_pred = np.array([classify_sample(x) for x in tqdm(test_features, desc="Classifying test samples")])

# 计算训练和测试的分类准确率
train_accuracy = np.mean(train_pred == train_labels)
test_accuracy = np.mean(test_pred == test_labels)
print(f"Training accuracy using minimum Mahalanobis distance classifier after feature extraction: {train_accuracy:.4f}")
print(f"Test accuracy using minimum Mahalanobobis distance classifier after feature extraction: {test_accuracy:.4f}")


Files already downloaded and verified
Files already downloaded and verified





Extracting features:   0%|          | 0/98 [00:00<?, ?it/s]


Extracting features:   1%|          | 1/98 [00:01<03:12,  1.98s/it]


Extracting features:   2%|▏         | 2/98 [00:03<02:50,  1.78s/it]


Extracting features:   3%|▎         | 3/98 [00:05<02:42,  1.71s/it]


Extracting features:   4%|▍         | 4/98 [00:06<02:38,  1.68s/it]


Extracting features:   5%|▌         | 5/98 [00:08<02:34,  1.66s/it]


Extracting features:   6%|▌         | 6/98 [00:10<02:32,  1.66s/it]


Extracting features:   7%|▋         | 7/98 [00:11<02:30,  1.66s/it]


Extracting features:   8%|▊         | 8/98 [00:13<02:38,  1.77s/it]


Extracting features:   9%|▉         | 9/98 [00:15<02:36,  1.76s/it]


Extracting features:  10%|█         | 10/98 [00:17<02:31,  1.72s/it]


Extracting features:  11%|█         | 11/98 [00:18<02:26,  1.69s/it]


Extracting features:  12%|█▏        | 12/98 [00:20<02:23,  1.67s/it]


Extracting features:  13%|█▎        | 13/98 [00:22<02:21,  1.66s/it]


Extracting features: 

KeyboardInterrupt: ignored

下面这个输入维度是64维

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy.spatial.distance import mahalanobis
from tqdm import tqdm
import timm

# 设置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载数据
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=512, shuffle=False)

# 使用timm库加载ResNet-18模型，并获取layer1输出
model = timm.create_model('resnet18', pretrained=True).to(device)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-4]).to(device)  # 选择到layer1的输出

feature_extractor.eval()

# 特征提取函数
def extract_features(loader, model):
    features = []
    labels = []
    with torch.no_grad():
        for data in tqdm(loader, desc="Extracting features"):
            inputs, label = data
            inputs = inputs.to(device)
            feature = model(inputs) # 获取模型的特征
            feature = torch.nn.functional.adaptive_avg_pool2d(feature, (1, 1))
            feature = feature.reshape(feature.size(0), -1)
            features.append(feature.cpu())
            labels.append(label)
    features = torch.cat(features, 0)
    labels = torch.cat(labels, 0)
    return features, labels

train_features, train_labels = extract_features(trainloader, feature_extractor)
test_features, test_labels = extract_features(testloader, feature_extractor)

# 计算每个类别的均值和协方差矩阵的逆
class_means = {}
class_inv_cov_matrices = {}
for c in range(10):
    data_c = train_features[train_labels == c]
    class_means[c] = torch.mean(data_c, dim=0)
    cov_matrix = torch.tensor(np.cov(data_c, rowvar=False))
    class_inv_cov_matrices[c] = torch.linalg.inv(cov_matrix)

# 使用最小Mahalanobis距离分类器进行分类
def classify(features, means, inv_cov_matrices):
    pred_labels = []
    for feature in tqdm(features, desc="Classifying"):
        distances = []
        for c in range(10):
            m_distance = mahalanobis(feature, means[c], inv_cov_matrices[c])
            distances.append(m_distance)
        pred_labels.append(distances.index(min(distances)))
    return torch.tensor(pred_labels)

train_pred = classify(train_features, class_means, class_inv_cov_matrices)
test_pred = classify(test_features, class_means, class_inv_cov_matrices)

train_acc = (train_pred == train_labels).float().mean().item()
test_acc = (test_pred == test_labels).float().mean().item()

print(f"Training Accuracy: {train_acc * 100:.2f}%")
print(f"Testing Accuracy: {test_acc * 100:.2f}%")


Files already downloaded and verified
Files already downloaded and verified


model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]





Extracting features:   0%|          | 0/98 [00:00<?, ?it/s]



Extracting features:   2%|▏         | 2/98 [00:00<00:07, 13.01it/s]



Extracting features:   4%|▍         | 4/98 [00:00<00:07, 13.05it/s]



Extracting features:   6%|▌         | 6/98 [00:00<00:06, 13.19it/s]



Extracting features:   8%|▊         | 8/98 [00:00<00:07, 12.68it/s]



Extracting features:  10%|█         | 10/98 [00:00<00:06, 13.00it/s]



Extracting features:  12%|█▏        | 12/98 [00:00<00:06, 12.99it/s]



Extracting features:  14%|█▍        | 14/98 [00:01<00:06, 13.06it/s]



Extracting features:  16%|█▋        | 16/98 [00:01<00:06, 12.90it/s]



Extracting features:  18%|█▊        | 18/98 [00:01<00:06, 12.81it/s]



Extracting features:  20%|██        | 20/98 [00:01<00:06, 12.61it/s]



Extracting features:  22%|██▏       | 22/98 [00:01<00:05, 12.77it/s]



Extracting features:  24%|██▍       | 24/98 [00:01<00:05, 12.96it/s]



Extracting features:  27%|██▋       | 26/98 [00:02<00:05, 12.96it/s]



E

Training Accuracy: 69.21%
Testing Accuracy: 62.20%


下面这个输出维度是256维

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy.spatial.distance import mahalanobis
from tqdm import tqdm
import timm

# 设置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载数据
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=512, shuffle=False)

# 使用timm库加载ResNet-50模型，并获取layer1输出
model = timm.create_model('resnet50', pretrained=True).to(device)
model.eval()

# 特征提取函数
def extract_features(loader, model):
    features = []
    labels = []
    with torch.no_grad():
        for data in tqdm(loader, desc="Extracting features"):
            inputs, label = data
            inputs = inputs.to(device)
            feature = model.forward_features(inputs)  # 获取模型的特征
            feature = torch.nn.functional.adaptive_avg_pool2d(feature, (1, 1))
            feature = feature.reshape(feature.size(0), -1)
            features.append(feature.cpu())
            labels.append(label)
    features = torch.cat(features, 0)
    labels = torch.cat(labels, 0)
    return features, labels

train_features, train_labels = extract_features(trainloader, model)
test_features, test_labels = extract_features(testloader, model)

# 计算每个类别的均值和协方差矩阵的逆
class_means = {}
class_inv_cov_matrices = {}
identity = torch.eye(train_features.size(1)).double() # 创建单位矩阵
regularization_term = 1e-6

for c in range(10):
    data_c = train_features[train_labels == c]
    class_means[c] = torch.mean(data_c, dim=0)
    cov_matrix = torch.tensor(np.cov(data_c, rowvar=False)) + regularization_term * identity
    class_inv_cov_matrices[c] = torch.linalg.inv(cov_matrix)

# 使用最小Mahalanobis距离分类器进行分类
def classify(features, means, inv_cov_matrices):
    pred_labels = []
    for feature in tqdm(features, desc="Classifying"):
        distances = []
        for c in range(10):
            m_distance = mahalanobis(feature, means[c], inv_cov_matrices[c])
            distances.append(m_distance)
        pred_labels.append(distances.index(min(distances)))
    return torch.tensor(pred_labels)

train_pred = classify(train_features, class_means, class_inv_cov_matrices)
test_pred = classify(test_features, class_means, class_inv_cov_matrices)

train_acc = (train_pred == train_labels).float().mean().item()
test_acc = (test_pred == test_labels).float().mean().item()

print(f"Training Accuracy: {train_acc * 100:.2f}%")
print(f"Testing Accuracy: {test_acc * 100:.2f}%")


Files already downloaded and verified
Files already downloaded and verified


Classifying:   4%|▍         | 2197/50000 [01:03<22:55, 34.74it/s]


KeyboardInterrupt: ignored

final

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from scipy.spatial.distance import mahalanobis
from tqdm import tqdm
import timm
import numpy as np

# 设置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载数据
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=512, shuffle=False)
train_labels = np.concatenate([data[1].numpy() for data in trainloader], 0)
test_labels = np.concatenate([data[1].numpy() for data in testloader], 0)

# 特征提取函数
def extract_pca_features(trainloader, testloader):
    train_data = np.concatenate([data[0].numpy().reshape(data[0].shape[0], -1) for data in tqdm(trainloader, desc="PCA - Processing Train Data")], 0)
    test_data = np.concatenate([data[0].numpy().reshape(data[0].shape[0], -1) for data in tqdm(testloader, desc="PCA - Processing Test Data")], 0)
    pca = PCA(0.9)
    train_features = pca.fit_transform(train_data)
    test_features = pca.transform(test_data)
    return train_features, test_features

def extract_lda_features(trainloader, testloader):
    train_data = np.concatenate([data[0].numpy().reshape(data[0].shape[0], -1) for data in tqdm(trainloader, desc="LDA - Processing Train Data")], 0)
    test_data = np.concatenate([data[0].numpy().reshape(data[0].shape[0], -1) for data in tqdm(testloader, desc="LDA - Processing Test Data")], 0)
    lda = LDA()
    train_features = lda.fit_transform(train_data, train_labels)
    test_features = lda.transform(test_data)
    return train_features, test_features

def extract_resnet_features(trainloader, testloader):
    model = timm.create_model('resnet18', pretrained=True, num_classes=0)
    model = model.to(device)
    model.eval()

    def get_features(loader, desc="ResNet - Processing Data"):
        features = []
        for data in tqdm(loader, desc=desc):
            inputs, _ = data
            inputs = inputs.to(device)
            feature = model.forward_features(inputs)
            feature = torch.nn.functional.adaptive_avg_pool2d(feature, (1, 1))
            feature = feature.reshape(feature.size(0), -1)
            features.append(feature.cpu().detach().numpy())
        return np.concatenate(features, 0)

    train_features = get_features(trainloader, desc="ResNet - Processing Train Data")
    test_features = get_features(testloader, desc="ResNet - Processing Test Data")
    return train_features, test_features

# 定义Mahalanobis分类器
def classify_with_mahalanobis(train_features, test_features, train_labels):
    means = {}
    inv_cov_matrices = {}
    regularization = np.eye(train_features.shape[1]) * 1e-6
    for c in tqdm(range(10), desc="Computing Class Means and Inverse Covariance"):
        data_c = train_features[train_labels == c]
        means[c] = np.mean(data_c, axis=0)
        cov_matrix = np.cov(data_c, rowvar=False) + regularization
        inv_cov_matrices[c] = np.linalg.inv(cov_matrix)

    predictions = []
    for feature in tqdm(test_features, desc="Classifying"):
        distances = [mahalanobis(feature, means[c], inv_cov_matrices[c]) for c in range(10)]
        predictions.append(np.argmin(distances))

    return np.array(predictions)

# 主程序
def main():
    methods = ["PCA", "LDA", "RESNET-18"]
    train_features_methods = [extract_pca_features, extract_lda_features, extract_resnet_features]

    for method, feature_func in zip(methods, train_features_methods):
        print(f"\nProcessing {method} features...")
        train_features, test_features = feature_func(trainloader, testloader)
        test_pred = classify_with_mahalanobis(train_features, test_features, train_labels)
        acc = np.mean(test_pred == test_labels)
        print(f"\n{method} Test Accuracy: {acc * 100:.2f}%")
        print("-------------------------")

if __name__ == "__main__":
    main()


Files already downloaded and verified
Files already downloaded and verified

Processing PCA features...


Classifying: 100%|██████████| 10000/10000 [00:02<00:00, 4823.01it/s]



PCA Test Accuracy: 9.31%
-------------------------

Processing LDA features...


Classifying: 100%|██████████| 10000/10000 [00:00<00:00, 13817.54it/s]



LDA Test Accuracy: 9.96%
-------------------------

Processing RESNET-18 features...


Classifying: 100%|██████████| 10000/10000 [00:12<00:00, 773.03it/s]


RESNET-18 Test Accuracy: 9.20%
-------------------------


输出64维

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy.spatial.distance import mahalanobis
from tqdm import tqdm
import timm

# 设置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载数据
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=512, shuffle=False)

# 使用timm库加载ResNet-18模型，并获取layer1输出
model = timm.create_model('resnet18', pretrained=True).to(device)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-4]).to(device)  # 选择到layer1的输出

feature_extractor.eval()

# 特征提取函数
def extract_features(loader, model):
    features = []
    labels = []
    with torch.no_grad():
        for data in tqdm(loader, desc="Extracting features"):
            inputs, label = data
            inputs = inputs.to(device)
            feature = model(inputs) # 获取模型的特征
            feature = torch.nn.functional.adaptive_avg_pool2d(feature, (1, 1))
            feature = feature.reshape(feature.size(0), -1)
            features.append(feature.cpu())
            labels.append(label)
    features = torch.cat(features, 0)
    labels = torch.cat(labels, 0)
    return features, labels

train_features, train_labels = extract_features(trainloader, feature_extractor)
test_features, test_labels = extract_features(testloader, feature_extractor)

# 计算每个类别的均值和协方差矩阵的逆
class_means = {}
class_inv_cov_matrices = {}
for c in range(10):
    data_c = train_features[train_labels == c]
    class_means[c] = torch.mean(data_c, dim=0)
    cov_matrix = torch.tensor(np.cov(data_c, rowvar=False))
    class_inv_cov_matrices[c] = torch.linalg.inv(cov_matrix)

# 使用最小Mahalanobis距离分类器进行分类
def classify(features, means, inv_cov_matrices):
    pred_labels = []
    for feature in tqdm(features, desc="Classifying"):
        distances = []
        for c in range(10):
            m_distance = mahalanobis(feature, means[c], inv_cov_matrices[c])
            distances.append(m_distance)
        pred_labels.append(distances.index(min(distances)))
    return torch.tensor(pred_labels)

train_pred = classify(train_features, class_means, class_inv_cov_matrices)
test_pred = classify(test_features, class_means, class_inv_cov_matrices)

train_acc = (train_pred == train_labels).float().mean().item()
test_acc = (test_pred == test_labels).float().mean().item()

print(f"Training Accuracy: {train_acc * 100:.2f}%")
print(f"Testing Accuracy: {test_acc * 100:.2f}%")


Files already downloaded and verified
Files already downloaded and verified


Classifying: 100%|██████████| 10000/10000 [00:05<00:00, 1926.80it/s]

Training Accuracy: 69.22%
Testing Accuracy: 62.20%


输出256维

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from scipy.spatial.distance import mahalanobis
from tqdm import tqdm
import timm

# 设置GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载数据
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=512, shuffle=False)

# 使用timm库加载ResNet-50模型，并获取layer1输出
model = timm.create_model('resnet50', pretrained=True).to(device)
model.eval()

# 特征提取函数
def extract_features(loader, model):
    features = []
    labels = []
    with torch.no_grad():
        for data in tqdm(loader, desc="Extracting features"):
            inputs, label = data
            inputs = inputs.to(device)
            feature = model.forward_features(inputs)  # 获取模型的特征
            feature = torch.nn.functional.adaptive_avg_pool2d(feature, (1, 1))
            feature = feature.reshape(feature.size(0), -1)
            features.append(feature.cpu())
            labels.append(label)
    features = torch.cat(features, 0)
    labels = torch.cat(labels, 0)
    return features, labels

train_features, train_labels = extract_features(trainloader, model)
test_features, test_labels = extract_features(testloader, model)

# 计算每个类别的均值和协方差矩阵的逆
class_means = {}
class_inv_cov_matrices = {}
identity = torch.eye(train_features.size(1)).double() # 创建单位矩阵
regularization_term = 1e-6

for c in range(10):
    data_c = train_features[train_labels == c]
    class_means[c] = torch.mean(data_c, dim=0)
    cov_matrix = torch.tensor(np.cov(data_c, rowvar=False)) + regularization_term * identity
    class_inv_cov_matrices[c] = torch.linalg.inv(cov_matrix)

# 使用最小Mahalanobis距离分类器进行分类
def classify(features, means, inv_cov_matrices):
    pred_labels = []
    for feature in tqdm(features, desc="Classifying"):
        distances = []
        for c in range(10):
            m_distance = mahalanobis(feature, means[c], inv_cov_matrices[c])
            distances.append(m_distance)
        pred_labels.append(distances.index(min(distances)))
    return torch.tensor(pred_labels)

train_pred = classify(train_features, class_means, class_inv_cov_matrices)
test_pred = classify(test_features, class_means, class_inv_cov_matrices)

train_acc = (train_pred == train_labels).float().mean().item()
test_acc = (test_pred == test_labels).float().mean().item()

print(f"Training Accuracy: {train_acc * 100:.2f}%")
print(f"Testing Accuracy: {test_acc * 100:.2f}%")


Files already downloaded and verified
Files already downloaded and verified



Extracting features: 100%|██████████| 98/98 [00:13<00:00,  7.20it/s]

Classifying train samples:   0%|          | 0/50000 [18:02<?, ?it/s]

Classifying: 100%|██████████| 10000/10000 [04:51<00:00, 34.34it/s]

Training Accuracy: 79.61%
Testing Accuracy: 49.57%
